In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!pip install keras-tuner
!pip install scikeras

In [67]:
import pandas as pd
import numpy as np
import joblib
import keras_tuner as kt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from keras_tuner.tuners import RandomSearch
from sklearn.preprocessing import MinMaxScaler

In [68]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataSet_FYP/new_student_study_schedule_modified.csv')

In [69]:
# Select relevant features and target
features = df[['Difficulty Level', 'Previous Mark', 'Latest Mark', 'Date Count']]
target = df['Study Time']

# Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Create sequences for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps), :]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 3
X_seq, y_seq = create_dataset(scaled_features, target.values, time_steps)

# Split the data keeping the temporal order for TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

In [70]:
# Define a function to create the LSTM model (for hyperparameter tuning)
# Define the LSTM model building function
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_seq.shape[1], X_seq.shape[2])))
    model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss='mean_squared_error')
    return model

In [71]:
# # Wrap the model with KerasRegressor for compatibility with scikit-learn
# model = KerasRegressor(build_model, epochs=10, batch_size=32, verbose=0)

In [72]:
# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='keras_lstm_tuning'
)

Reloading Tuner from my_dir/keras_lstm_tuning/tuner0.json


In [73]:
# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

In [74]:
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [75]:
# Perform hyperparameter tuning
for train_index, val_index in tscv.split(X_seq):
    X_train, X_val = X_seq[train_index], X_seq[val_index]
    y_train, y_val = y_seq[train_index], y_seq[val_index]
    tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

In [76]:
# Retrieve the best model. Here, you no longer use `get_best_models()` because it's deprecated.
best_hp = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hp)

In [77]:
# Evaluate the best model with cross-validation
mae_scores, mse_scores, rmse_scores = [], [], []
for train_index, test_index in tscv.split(X_seq):
    X_train, X_test = X_seq[train_index], X_seq[test_index]
    y_train, y_test = y_seq[train_index], y_seq[test_index]

In [78]:
# Train the best model
best_model.fit(X_seq, y_seq, epochs=50, validation_split=0.2, callbacks=[early_stopping], verbose=0)

In [79]:
# Predict on the test set
y_pred = best_model.predict(X_seq)

9471/9471 [==============================] - 48s 5ms/step


In [80]:
# Calculate the evaluation metrics
mae_score = mean_absolute_error(y_seq, y_pred)
mse_score = mean_squared_error(y_seq, y_pred)
rmse_score = np.sqrt(mse_score)

# Print the evaluation metrics
print(f'Mean MAE: {mae_score}')
print(f'Mean MSE: {mse_score}')
print(f'Mean RMSE: {rmse_score}')

Mean MAE: 39.72196849697467
Mean MSE: 2112.5025590179607
Mean RMSE: 45.96196861556259


In [81]:
# Save the best model
best_model.save('/content/drive/MyDrive/Colab Notebooks/Models/best_lstm_model.h5')
# Save the scaler object
joblib.dump(scaler, '/content/drive/MyDrive/Colab Notebooks/Models/scaler_X.pkl')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/MyDrive/Colab Notebooks/Models/scaler_X.pkl']